In [509]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import plotly.express as px

In [510]:
model = tf.keras.models.load_model('predict_model.keras')
#model = pickle.load(open('linear_regressor.pkl', 'rb'))

In [511]:
scaler = pickle.load(open('scaler.pkl', 'rb'))

In [512]:
columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number', 'CO(mg/m^3)_final']
#mae: 0.1278

In [513]:
output_column = 'CO(mg/m^3)_final'

In [514]:
input_columns = ['CO(mg/m^3)_initial', 'Volume(m^3)', 'N_people_MA_prev_10min',	'Ambient-Air-Pump(L/min)', 'Ambient-Air-Pump_power(%)',	'Ambient-Air-Pump_number']

In [515]:
CO = 0.018
volume = 30
people_now = 1
people_tn = [people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now, people_now]
pumps_l_min = 566
pump_power = 50.0
n_pumps = 8
index = 0

In [516]:
def create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index):
    df = pd.DataFrame({
        'CO(mg/m^3)_initial':CO,
        'Volume(m^3)':volume, 
        'N_people_MA_prev_10min': sum(people_tn) / len(people_tn),	
        'Ambient-Air-Pump(L/min)': pumps_l_min, 
        'Ambient-Air-Pump_power(%)': pump_power,
        'Ambient-Air-Pump_number': n_pumps
    }, index=[index])
    return df

In [517]:
df_sim = pd.DataFrame(columns = columns)
for i in range(10):
    index = i
    df_t = create_df_input(CO, volume, people_tn, pumps_l_min, pump_power, n_pumps, index)
    df_t_scaled = scaler.transform(df_t)
    df_t_scaled = pd.DataFrame(df_t_scaled, columns=input_columns)
    CO = model.predict(df_t_scaled)[0][0]
    CO += (np.random.rand(1)[0]*2 -1)* 0.01 * CO
    df_t["CO(mg/m^3)_final"] = CO
    df_sim = pd.concat([df_sim,df_t])

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 18ms/step


/tmp/ipykernel_15022/235172288.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



1/1 [==============================] - 0s 12ms/step


In [518]:
fig = px.line(df_sim["CO(mg/m^3)_final"])
fig.show()

In [519]:
df_sim

,CO(mg/m^3)_initial,Volume(m^3),N_people_MA_prev_10min,Ambient-Air-Pump(L/min),Ambient-Air-Pump_power(%),Ambient-Air-Pump_number,CO(mg/m^3)_final
0,0.018000,30,1.0,566,50.0,8,0.018934
1,0.018934,30,1.0,566,50.0,8,0.020497
2,0.020497,30,1.0,566,50.0,8,0.024259
3,0.024259,30,1.0,566,50.0,8,0.039378
4,0.039378,30,1.0,566,50.0,8,0.168360
5,0.168360,30,1.0,566,50.0,8,0.256307
6,0.256307,30,1.0,566,50.0,8,0.339225
7,0.339225,30,1.0,566,50.0,8,0.474023
8,0.474023,30,1.0,566,50.0,8,0.563268
9,0.563268,30,1.0,566,50.0,8,0.602023
